In [1]:
import numpy as np
import pandas as pd
import config_final
import requests
from bs4 import BeautifulSoup as BS


In [2]:
config_final.user

'munzma'

# Webscrape URL List for Bill Summaries

In [3]:
df = pd.read_csv('bills93-114.csv', sep=";", encoding='latin-1')
df.dropna(axis=0, subset = ['Cosponsr', 'URL'], inplace=True)
urls = df[['BillID','URL']]
# df = df.set_index('BillID')

print(df.shape)

(294336, 48)


In [4]:
urls['URL'] = urls['URL'].astype('str') 

/Users/melissamunz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [5]:
df.head()

,BillID,BillNum,BillType,Chamber,Cong,Cosponsr,IntrDate,Mult,MultNo,PLaw,...,PLawDate,PLawNum,ImpBill,SubChRef,SubRankRef,Majority,ReportH,ReportS,Major,Minor
0,93-HCONRES-1,1,hconres,0,93,0.0,1973-01-03,0.0,1.0,0,...,NaN,NaN,0.0,NaN,NaN,1,0,0,NaN,NaN
1,93-HCONRES-2,2,hconres,0,93,0.0,1973-01-03,0.0,1.0,0,...,NaN,NaN,0.0,NaN,NaN,1,0,0,NaN,NaN
2,93-HCONRES-3,3,hconres,0,93,0.0,1973-01-03,0.0,1.0,0,...,NaN,NaN,0.0,NaN,NaN,1,0,0,NaN,NaN
3,93-HCONRES-4,4,hconres,0,93,0.0,1973-01-03,0.0,1.0,0,...,NaN,NaN,0.0,NaN,NaN,0,0,0,NaN,NaN
4,93-HCONRES-5,5,hconres,0,93,0.0,1973-01-03,0.0,1.0,0,...,NaN,NaN,0.0,NaN,NaN,0,0,0,NaN,NaN


## Create Scraper

In [6]:
from bs4 import NavigableString

def scraper(url):
       
    response = requests.get(url)
    soup = BS(response.content, "html.parser")
    soup.prettify() 
    
    bill_summary = soup.find('div', id="bill-summary")
    paragraphs = bill_summary.find_all('p')
    
    if len(paragraphs) > 1:
        return paragraphs[1].get_text(strip=True)
    else:
        node = soup.find('div', id="bill-summary").contents[-1]
#         print(type(node))
        if type(node) == NavigableString:
            return node.strip()
        else:
            print(url) 
            return ''

                 
    

In [7]:
#Get the summary into dataframe!

# df['Summary'] = df.apply (lambda row: scraper(row['URL']), axis=1)

In [8]:
urls.head()

,BillID,URL
0,93-HCONRES-1,https://www.congress.gov/bill/93th-congress/ho...
1,93-HCONRES-2,https://www.congress.gov/bill/93th-congress/ho...
2,93-HCONRES-3,https://www.congress.gov/bill/93th-congress/ho...
3,93-HCONRES-4,https://www.congress.gov/bill/93th-congress/ho...
4,93-HCONRES-5,https://www.congress.gov/bill/93th-congress/ho...


In [9]:
#Test - Convert to dictionary format
final_summaries = df.to_dict(orient = 'records')

## Enter into Database

In [10]:
from sodapy import Socrata
import sqlalchemy as db

In [11]:
def query(q):
    try:
        return pd.read_sql_query(q, engine)
    # Pass when no data is returned    
    except ResourceClosedError:
        pass

def query_list(col, table, distinct = True):
    elts = ['SELECT',
            'DISTINCT' if distinct else '',
            col,
            'FROM',
            table]
    query_str = ' '.join(elts)
    df = query(query_str)
    l = df.iloc[:,0].tolist()
    return l

**Create**


In [12]:
df.shape

(294336, 48)

**Cleaning**

In [13]:
df['Class'].fillna(0, inplace=True)

df['PooleID'].fillna(0, inplace=True)

df['PLawDate'].fillna(0, inplace=True)

df['PLawNum'].fillna(0, inplace=True)

df['Major'].fillna(0, inplace=True)

df['Minor'].fillna(0, inplace=True)

In [14]:
df.isnull().sum()

BillID            0
BillNum           0
BillType          0
Chamber           0
Cong              0
Cosponsr          0
IntrDate          0
Mult              0
MultNo            0
PLaw              0
Private           0
Title             0
Veto              0
Class             0
ComC              0
ComR              0
CumHServ          0
CumSServ          0
Delegate          0
District          0
DW1             215
FrstConH          0
FrstConS          0
Gender            0
MemberID          0
MRef              0
NameFirst         0
NameFull          0
NameLast          0
Party             0
PooleID           0
Postal            0
State             0
URL               0
ChRef             0
RankRef           0
PassH             0
PassS             0
PLawDate          0
PLawNum           0
ImpBill           0
SubChRef      53327
SubRankRef    53327
Majority          0
ReportH           0
ReportS           0
Major             0
Minor             0
dtype: int64

**Create**

In [15]:
#instantiate the connection and table

engine = db.create_engine(f'mysql+mysqlconnector://{config_final.user}:{config_final.password}@{config_final.host}/con_bills')

connection = engine.connect()
metadata=db.MetaData()

In [16]:
#Create Table 
bills_table = db.Table('bills', metadata,               
    db.Column('BillID', db.String(40), primary_key=True),
    db.Column('BillType', db.String(20)),
    db.Column('Chamber', db.Boolean()),
    db.Column('Cong', db.Integer()),
    db.Column('Cosponsr', db.Float()),
    db.Column('IntrDate', db.DateTime()),
    db.Column('Mult', db.Boolean()),
    db.Column('PLaw', db.Boolean()),
    db.Column('Title', db.String(500)),
    db.Column('Veto', db.Boolean()),
    db.Column('Class', db.Float()),
    db.Column('District', db.Float()),
    db.Column('FrstConH', db.Float()),
    db.Column('FrstConS', db.Float()),
    db.Column('Gender', db.Boolean()),
    db.Column('MRef', db.Boolean()),
    db.Column('NameFull', db.String(40)),
    db.Column('Party', db.Float()),
    db.Column('Postal', db.String(10)),
    db.Column('PassH', db.Boolean()),
    db.Column('PassS', db.Boolean()),
    db.Column('PLawDate', db.DateTime()),
    db.Column('PLawNum', db.String(40)),
    db.Column('ImpBill', db.Boolean()),
    db.Column('Majority', db.Boolean()),
    db.Column('Major', db.Float()),
    db.Column('Minor', db.Float()),
    db.Column('Summary', db.String(2000))
                         
)

In [17]:
metadata.create_all(engine)

In [18]:
df.head()

,BillID,BillNum,BillType,Chamber,Cong,Cosponsr,IntrDate,Mult,MultNo,PLaw,...,PLawDate,PLawNum,ImpBill,SubChRef,SubRankRef,Majority,ReportH,ReportS,Major,Minor
0,93-HCONRES-1,1,hconres,0,93,0.0,1973-01-03,0.0,1.0,0,...,0,0,0.0,NaN,NaN,1,0,0,0.0,0.0
1,93-HCONRES-2,2,hconres,0,93,0.0,1973-01-03,0.0,1.0,0,...,0,0,0.0,NaN,NaN,1,0,0,0.0,0.0
2,93-HCONRES-3,3,hconres,0,93,0.0,1973-01-03,0.0,1.0,0,...,0,0,0.0,NaN,NaN,1,0,0,0.0,0.0
3,93-HCONRES-4,4,hconres,0,93,0.0,1973-01-03,0.0,1.0,0,...,0,0,0.0,NaN,NaN,0,0,0,0.0,0.0
4,93-HCONRES-5,5,hconres,0,93,0.0,1973-01-03,0.0,1.0,0,...,0,0,0.0,NaN,NaN,0,0,0,0.0,0.0


In [39]:
#Test - Convert to dictionary format
final = df.to_dict(orient = 'records')

In [40]:
reversed(final)


In [ ]:
# for bill in final:
#     bill['Summary'] = scraper(bill['URL'])
#     query = db.insert(bills_table)
#     connection.execute(query, bill)

In [ ]:
final[3]

In [ ]:
# query = db.insert(bills)

In [ ]:
# connection.execute(query, final)

In [ ]:
# Stopped at 17778

#use reversed(final) to start from the other side.